# Exploratory Analysis (House Prices)

## Description
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Una primera aproximacio per mirar les variables que tenim (n'hi han moltes).   
  --> histogrames ?  
  --> comparant train y test?   

la idea seria fer una primera aproximacio amb model de LR Multinomial, amb un backward elimination per identificar variables destacades y provar validacion